# Leakage of flow in Immersed Boundary Methods

This notebook will be used to develop the functions required to calculate the
leakage of flow in Immersed Boundary Methods. The leakage is defined as the flow
that goes through the immersed boundary and is not accounted for in the
simulation. This is a common problem in Immersed Boundary Methods, and it is
important to quantify it in order to understand the accuracy of the simulation.

The leakage is calculated as the average flow velocity through the immersed
boundary. This is done by probing the flow velocity at in the immersed region
and calculating the average velocity.

In [ ]:
# Define the test examples and flags

if __name__ == "__main__":
    debug = True
    root_dir = "../../"
    data_location = root_dir + "results/hpc/cylinder_benchmark/explicit_100/"
    data_file = data_location + "leakage.csv"


In [ ]:
# Setup imports

import sys
import os

try:
    import matplotlib.pyplot as plt
    import numpy as np
except:
    %pip install matplotlib numpy
    import matplotlib.pyplot as plt
    import numpy as np

try:
    from nekotop import Probes
except:
    sys.path.append(os.path.abspath(root_dir + 'scripts/modules/'))
    from nekotop import Probes


In [ ]:
if __name__ == "__main__":

    if "probes" not in locals() or debug:
        probes = Probes(data_file)

    # Compute the mean velocity for each timestep
    u = probes.fields["u"]
    v = probes.fields["v"]

    if "velocity" not in locals() or debug:
        velocities = np.sqrt(u**2 + v**2)

## Computing average across the cylinder

Points are located in a cylinder, however we should compress it down to a disk,
averaging across the z coordinate of the cylinder. However, the velocity array
is currently stored as N_points x N_timesteps, so we need to detect points that
are in the same z plane and average them.


In [ ]:
# The points are ordered in the following way:

if __name__ == "__main__":
    points_2d = np.asarray((probes.points[:, 0], probes.points[:, 1])).T
    _, index = np.unique(points_2d, return_index=True, axis=0)

    # Now we can average the velocities
    points = probes.points
    points_2d = points[index, :2]
    velocities_2d = np.zeros((len(index), len(probes.times)))

    for i in range(len(index)):
        p2 = points_2d[i]
        n = 0
        for j in range(len(points)):
            p3 = points[j]
            if p2[0] == p3[0] and p2[1] == p3[1]:
                velocities_2d[i, :] += velocities[j, :]
                n += 1

        velocities_2d[i, :] /= n

In [ ]:
# Define a manim scene for the disk of points and velocities

if __name__ == "__main__":

    import manim

    threshold = 1e-2

    manim.config.media_dir = "manim_media"
    manim.config.verbosity = "WARNING"
    manim.config.progress_bar = "none"
    manim.config.disable_caching = debug

    # Set the resolution and field of view
    manim.config.pixel_height = 400
    manim.config.pixel_width = 400
    manim.config.frame_width = 1.1
    manim.config.frame_height = 1.1

    class Example(manim.Scene):

        def construct(self):
            dots = manim.VGroup()

            for ip in range(len(points_2d)):
                i = index[ip]
                point = points[i, :]
                c = (velocities_2d[ip, 250] - threshold) / threshold
                col = np.asarray([1.0, 1.0 - c, 1.0 - c, 1.0])
                dots += manim.Dot(point=point, color=col, radius=0.01)

            self.add(dots)

    %manim Example

In [ ]:
def velocity_statistics(velocities, start=0, end=-1):
    """
    Compute the statistics of the velocity field
    """

    mean = np.mean(velocities[:, start:end], axis=0)
    std = np.std(velocities[:, start:end], axis=0)
    max = np.max(velocities[:, start:end], axis=0)
    min = np.min(velocities[:, start:end], axis=0)

    return mean, std, max, min


In [ ]:
# Draw statistics based on the velocities at the probe points

if __name__ == "__main__":

    start = 0
    end = 250

    mu, _, vel_max, vel_min = velocity_statistics(velocities, start, end)

    # Plot the probed data
    fig, ax = plt.subplots(1, 1, figsize=(10, 5))
    print("Smallest max velocity: ", np.min(vel_max))
    print("Min velocity: ", np.min(vel_min))

    # Animate the velocity as color for each probe point
    time = probes.times[start:end]

    # Plot the velocity
    ax.plot(time,
            mu,
            label="Mean velocity",
            color="green",
            linestyle="-",
            linewidth=2)
    ax.plot(time,
            vel_max,
            label="Max velocity",
            color="red",
            linestyle="--",
            linewidth=2)
    ax.plot(time,
            vel_min,
            label="Min velocity",
            color="blue",
            linestyle="--",
            linewidth=2)

    ax.hlines(threshold,
              time[0],
              time[-1],
              color="black",
              linestyle="-.",
              label="Threshold")

    ax.set_xlabel("Time")
    ax.set_ylabel("Velocity")
    ax.legend()

    ax.set_yscale("log")

    plt.show()


In [ ]:
def leakage_benchmark(data_file, start=0, end=-1):
    """
    Compute the leakage for the benchmark
    """

    probes = Probes(data_file)

    # Compute the mean velocity for each timestep
    u = probes.fields["u"][:, start:end]
    v = probes.fields["v"][:, start:end]
    velocities = np.sqrt(u**2 + v**2)

    # The points are ordered in the following way:
    points_2d = np.asarray((probes.points[:, 0], probes.points[:, 1])).T
    _, index = np.unique(points_2d, return_index=True, axis=0)

    # Now we can average the velocities
    points = probes.points
    points_2d = points[index, :2]
    velocities_2d = np.zeros((len(index), len(probes.times[start:end])))

    for i in range(len(index)):
        p2 = points_2d[i]
        n = 0
        for j in range(len(points)):
            p3 = points[j]
            if p2[0] == p3[0] and p2[1] == p3[1]:
                velocities_2d[i, :] += velocities[j, :]
                n += 1

        velocities_2d[i, :] /= n

    # Compute the statistics of the velocity field
    vel_mean = np.mean(velocities_2d, axis=0)
    vel_std = np.std(velocities_2d, axis=0)
    vel_max = np.max(velocities_2d, axis=0)
    vel_min = np.min(velocities_2d, axis=0)

    result = {
        "times": probes.times[start:end],
        "mean": vel_mean,
        "std": vel_std,
        "max": vel_max,
        "min": vel_min
    }

    return result
